In [ ]:
import numpy as np
import seaborn as sb
import pandas
import sys
import itertools
import matplotlib.pyplot as plt
import nltk
import csv
import datetime
%matplotlib notebook

# Recurrent Neural Networks (RNNs)

The following is taken from the excellent tutorial on RNNs and adapted a bit.

http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-2-implementing-a-language-model-rnn-with-python-numpy-and-theano/

Our goal is to learn dependencies between words from a corpus given a recurrent neural network architecture.

## Step 1: Reading in the corpus

In the following, we are going to use a Shakespeare corpus (that only includes his plays, not the sonnets) to train our RNN. 

In order to get our network to properly train sentences, we are going to prepend sentence start tokens and append sentence end tokens to each of the corpus's sentences.

We are going to make use of the natural language toolkit to help with some processing.

In order to get this, you need to `pip3 install nltk` and then type from a python console `ntlk.download()` and select the popular packages for downloading.

In [ ]:
vocabularySize = 8000
unknownToken = "UNK"
sentenceStartToken = "S_START"
sentenceEndToken = "S_END"
 
# Read the data and append SENTENCE_START and SENTENCE_END tokens
print("Reading shakespeare plays...")
with open('data/shakespeare_plays.txt') as f:
    # split the data into sentences using nltk magic
    # this simply iterates through each line, lowercases it, and then
    # sends it to nltk, which splits the lines into sentences according
    # to English punctuation rules (i.e., .!?) and also handles problem cases
    sentences = itertools.chain(*[nltk.sent_tokenize(x.lower()) for x in f])
    # for each of the sentences we prepend S_START and
    # and append S_END tokens for our network to learn
    sentences = ["%s %s %s" % (sentenceStartToken, x, sentenceEndToken) for x in sentences]
print("read {:d} sentences".format(len(sentences)))

Now we are going to use ntlk to tokenize our sentences - tokenize in this case means to split it into words and sentence markers (such as ,.! and so on). In our case, we would like to get words.

In [ ]:
# now get words from the sentences - nltk uses much smarter methods
# to do this than our crude regular expressions from the first assignment
# but: if also takes a lot longer...
tokenizedSentences = [nltk.word_tokenize(sent) for sent in sentences]
 
# now we determine the unigram
unigram = nltk.FreqDist(itertools.chain(*tokenizedSentences))
print ("found {:d} unique words".format(len(unigram.items())))


## Step 2: Preparing the data structures

The RNN will not directly work on words, but will process each word as its index in the corresponding unigram of the text. 

So, we will use nltk to give us the unigram and the indices, and then create a variable `index2word` and `word2index` that will convert back and forth between the two representations.

The RNN will be trained on the output of `word2index`.

In [ ]:
# use only the vocabularySize most common words - yes, nltk has a
# helper function for that, too ^^
vocab = unigram.most_common(vocabularySize-1)
# create index2word and word2index indexing vectors that we need later
index2word = [x[0] for x in vocab]
# don't forget our unknown token:
index2word.append(unknownToken)
# and reverse index2word:
word2index = dict([(w,i) for i,w in enumerate(index2word)])
 
print("We choose a vocabulary size of {:d}".format(vocabularySize))
print("The least frequent word in the vocabulary is '{:s}' and appeared {:d} times.".format(vocab[-1][0], vocab[-1][1]))
print("The most frequent non start/end token word in the vocabulary is '{:s}' and appeared {:d} times.".format(vocab[2][0], vocab[2][1]))


Note that we restricted our vocabulary to 8000, so the unigram will contain words that our not in our dictionary. 

We have to replace these words by the UNK token in order to deal with them during training and testing.

In [ ]:
# now we need to go back and replace all words that are NOT
# in the vocabulary with the UNK token!
for i, sent in enumerate(tokenizedSentences):
    tokenizedSentences[i] = [w if w in word2index else unknownToken for w in sent]
    
print("\nExample sentence: '{:s}'".format(sentences[0]))
print("\nExample sentence after Pre-processing:",tokenizedSentences[0])


## Step 3: Create training data

This is the easiest part. We simply get all the words and put them into an array. 

In [ ]:
# create training data for the network
Xtrain = np.asarray([[word2index[w] for w in sentence[:-1]] for sentence in tokenizedSentences])
ytrain = np.asarray([[word2index[w] for w in sentence[1:]] for sentence in tokenizedSentences])

# show examples sentences for X and y 
for i in Xtrain[40]:
    print(index2word[i])
for i in ytrain[40]:
    print(index2word[i])

## Step 4: The RNN

Now for the actual beef of this. In the following, we will create an RNN-class that handles all the training for us.

The reason for doing a class is that this nicely capsulates all the functionality for us. Also, all current python interfaces to neural network tools (such as tensorflow, Theano, etc.) use class-based approaches, so that we can get some feeling here as well.

### Step 4.1: Initialization
Let's first add the constructor to the class:

In [ ]:
class RNN:
    # the constructor of the class - this function gets automatically 
    # called if you make a new instance of the class
    
    # the first variable is the name under which you refer to the
    # current instance itself and is usually named "self"
    
    # wordDim:      the number words in our dictionary
    # hiddenDim:    the number of hidden states or "memory"
    # bpttTruncate: the number of time-steps we can go back
    def __init__(self, wordDim, hiddenDim=100, bpttTruncate=4):
        # assign the instance variables
        self.wordDim = wordDim
        self.hiddenDim = hiddenDim
        self.bpttTruncate = bpttTruncate
        # initialize the network weights with small random numbers
        # there is a bit of "witchcraft" here in how you initialize these
        # for now, let's just say that small, random values are fine, but that
        # you can do better with [-1/sqrt(n),1/sqrt(n)]
        self.U = np.random.uniform(-np.sqrt(1./wordDim), np.sqrt(1./wordDim), (hiddenDim, wordDim))
        self.V = np.random.uniform(-np.sqrt(1./hiddenDim), np.sqrt(1./hiddenDim), (wordDim, hiddenDim))
        self.W = np.random.uniform(-np.sqrt(1./hiddenDim), np.sqrt(1./hiddenDim), (hiddenDim, hiddenDim))

### Step 4.2: The forward propagation and the loss function
Now let's define one forward pass through the RNN. For this, we need the output of the final layer, which is an array of probabilities for each of the words in our dictionary. 

There are many ways to define this, but a popular loss is the softmax-loss, which uses the softmax-function $S$ that takes as input a vector $\vec{x}$ of dimension $d$ and returns as output:

$S(\vec{x}) = \frac{e^{\vec{x}}}{\sum_{k=1}^{D}e^{x_k}}$

One easy view of the effects of this function is that it stretches the contrast of the input, such that high elements of the vector in the input will be even higher in the output. The output is normalized to be between 0 and 1, and that gives us basically a probabilistic output!

Because of numerical issues with the exponential function, large elements will lead to overflow (think $e^{1000}$), so people use the fact that the function $S(\vec{x})=S(C\vec{x})$ for an arbitrary $C$, which we can freely choose. In practice $C=max_k(x_k)$

In [ ]:
# output loss for the final layer
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)

Now for the actual forward pass.

In [ ]:
# one forward pass
def forwardProp(self, x):
    # how many time steps we have - this is the number of
    # words in the sentence!
    T = len(x)
    # we save all hidden states in s, because we need them later
    # we reserve one additional element for the initial hidden state
    s = np.zeros((T + 1, self.hiddenDim))
    
    # the outputs for each time step - each time step will predict
    # probabilities for all wordDim words!!!
    o = np.zeros((T, self.wordDim))
    # now go through each time step
    for t in np.arange(T):
        # the current hidden state is the activation function of
        # the input, which is the previous hidden state multiplied
        # by W and the addition of the current variable weighted by U
        
        # in order to multiply U with x[t], we would need to do some
        # sort of one-hot encoding of x[t]! This would be done in 
        # standard NN packages
        # but we can also use a trick: we can simply index into 
        # our weight array U with x[t], which will be the same 
        # as multiplying U with a one-hot vector
        
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

# we only do this here, since we left the "class" scope and are
# doing an "outside" decleration of that class functionality
RNN.forwardProp=forwardProp

### Step 4.3: Predicting something with the RNN
The output of the RNN is a large vector of probabilities, with one element for each of our dictionary words.

So, if we want to predict the "best" word, we simply need to select the index with the highest probability. 

In [ ]:
def predict(self, x):
    # do forward propagation and the index of the highest score
    o, _ = self.forwardProp(x)
    return np.argmax(o, axis=1)
 
RNN.predict = predict

### Testing the forward pass

Let's choose our example sentence and push it through. 

In [ ]:
np.random.seed(1)
# init
model = RNN(vocabularySize)
# do one pass with an example sentence that has 11 words
o, s = model.forwardProp(Xtrain[40])
# so our output will have probabilities for each of the 11 words for
# all entries in our dictionary
print(o.shape)
print(o)
# here are the hidden states [we have one more to model the initial state]
print(s.shape)

These probabilities are completely random, of course, since we actually have not trained the model!

### Predicting something with the model
Let's use that example sentence to predict the words that we would get and compare that to the actual words that we wanted to have.

In [ ]:
predictions = model.predict(Xtrain[40])
print(predictions.shape)
print(predictions)
for idx,i in enumerate(predictions):
    print('predicted:',index2word[i],' wanted:',index2word[ytrain[40][idx]])

As we can see, we don't have much of a match, which is understandable, since all the weights are currently random.

### Step 4.4 Determine loss

The final layer calculates the output, which consists of the probabilities for each word in the dictionary. Now, to determine the full loss, we need to of course do this **for each** of the sentences and then sum up the log of the probabilities to keep things small and easy.

In [ ]:
def calculateTotalLoss(self, x, y):
    # initialize loss
    L = 0
    # go through each sentence of the target
    for i in np.arange(len(y)):
        # do a forward pass and get the output (and the state for which we do not care here)
        o, _ = self.forwardProp(x[i])
        # now we check the output for those words that are contained
        # in the target sentence:
        correctWordProbabilities = o[np.arange(len(y[i])), y[i]]
        # this tells us how wrong we are, so we add that to the loss
        L += -1 * np.sum(np.log(correctWordProbabilities))
    return L

# this one is easy, we just some up all the words we have in the 
# target set 
def calculateLoss(self, x, y):
    # total loss divided by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return self.calculateTotalLoss(x,y)/N
 
RNN.calculateTotalLoss = calculateTotalLoss
RNN.calculateLoss = calculateLoss


Let's test this loss. Running this on the full dataset will be very slow, so here we only use a subset to show that our initial loss is close to the random prediction:

In [ ]:
print("Expected Loss for random predictions: {:f}".format(np.log(vocabularySize)))
# running this on the full data set is prohibitively slow
# so just use the first 100 to "approximate" 
print("Actual loss: {:f}".format(model.calculateLoss(Xtrain[:100], ytrain[:100])))

### Step 4.5: Backpropagation through time (BPTT)

Here comes the second piece of magic and the core of RNNs: the back-propagation through time.

If you remember our discussion on training standard multi-layer networks, you also remember backpropagation.

Here, the algorithm is exactly the same with the added twist that time gets unfolded - note that we would need to do this for the full length of the training data! 

The actual weight updates are calculated in the exact same way as for the standard backprop algorithm.

You initialize with the output of the last layer and then push back through all hidden layers.

Note that a full understanding of this algorithm requires the calculation of a rather lengthy set of derivatives. For a full explanation and derivation, see here:
https://github.com/go2carter/nn-learn/blob/master/grad-deriv-tex/rnn-grad-deriv.pdf


Note also that there are many more methods to implement this, including shortening the loop:
https://gist.github.com/m-liu/36c5e78d98c92fd8fa0c0f1efe2269c4



In [ ]:
def bptt(self, x, y):
    # number of "time steps" - again this is the length of the sentence!
    T = len(y)
    # push everything through the network once
    o, s = self.forwardProp(x)
    # these store the gradients
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    # final layer initialization: 
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # now we take a look at each output
    # and go backwards through each word in the sentence
    for t in np.arange(T)[::-1]:
        # update of loss with respect to hidden node
        dLdV += np.outer(delta_o[t], s[t].T)
        # now calculate first delta
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # now unfold network and go back through time 
        # we do a maximum of self.bpttTruncate time travel steps
        # the max's are there to avoid boundary effects
        # the higher this parameter, the better in "theory"
        for bpttStep in np.arange(max(0, t-self.bpttTruncate), t+1)[::-1]:
            #print("bptt: step t={:d} bptt step={:d} ".format(t, bpttStep))
            # update W
            dLdW += np.outer(delta_t, s[bpttStep-1])
            # update U
            dLdU[:,x[bpttStep]] += delta_t
            # change delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bpttStep-1] ** 2)
    return [dLdU, dLdV, dLdW]
 
RNN.bptt = bptt

### Step 4.6: Make training

In order to demonstrate the BPTT, we implement a function that only does one update step of all weights.

This is a very simple gradient descent update on all three parameters.

In [ ]:
def gd1step(self,Xtrain,ytrain,learningRate):
    # first calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(Xtrain, ytrain)
    # then change parameters according to gradients and learning rate
    self.U -= learningRate * dLdU
    self.V -= learningRate * dLdV
    self.W -= learningRate * dLdW
    
RNN.gd1step = gd1step

Now for the actual training. Here we put in the gradient descent. 

In [ ]:
# gradient descent training
# - model: The RNN model instance
# - Xtrain: The training data set [words x sentences]
# - ytrain: The target training data [words [shifted by one] x sentences]
# - learningRate: Initial learning rate for SGD
# - nEpoch: Number of times to iterate through the complete dataset
# - evaluateLossEvery: Evaluate the loss every epochs
def train(model, Xtrain, ytrain, learningRate=0.005, nEpoch=100, evaluateLossEvery=5):
    # keep track of the losses so we can plot them later
    losses = []
    numExamplesSeen = 0
    # loop through all epochs
    for epoch in range(nEpoch):
        # do we want to evaluate the loss?
        # caution: this can be expensive!
        if (epoch % evaluateLossEvery == 0 and epoch>0):
            loss = model.calculateLoss(Xtrain, ytrain)
            losses.append((numExamplesSeen, loss))
            time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print ("{:s}: Loss after numExamplesSeen={:d} epoch={:d}: {:f}".format(time, numExamplesSeen, epoch, loss))
            # Adjust the learning rate if loss increases
            #if (len(losses) >= 1 and losses[-1][1] >= losses[-2][1]):
            #    learningRate = learningRate * 0.5 
            #    print("Setting learning rate to {:f}".format(learning_rate))
            sys.stdout.flush()
        # for each training sentence...
        for i in range(len(ytrain)):
            # do one gradient step
            model.gd1step(Xtrain[i], ytrain[i], learningRate)          
            numExamplesSeen += 1
            if (i%1000 ==0):
                print(numExamplesSeen)

### Step 5: Testing

Let's test how fast one step is for one sentence.

In [ ]:
np.random.seed(1)
model = RNN(vocabularySize)
%timeit model.gd1step(Xtrain[40], ytrain[40], 0.005)

Ouch. Now imagine that for one whole epoch, where we have 160K sentences, which means roughly one hour to go through our full training set! In order to do good training, we should probably do a few 10s of epochs!!  

In [ ]:
np.random.seed(1)
# Train on a small subset of the data to see what happens
model = RNN(vocabularySize)
losses = train(model, Xtrain[:10], ytrain[:10], nEpoch=1000, evaluateLossEvery=100)

In [ ]:
indexToTest=5
predictions = model.predict(Xtrain[indexToTest])
print(predictions.shape)
print(predictions)
for idx,i in enumerate(predictions):
    print('predicted:',index2word[i],' wanted:',index2word[ytrain[indexToTest][idx]])

Much better :)

In [ ]:
def generateSentence(model):
    # since the model predict indices, we are going to get
    # index values into a variable called newSentence
    
    # sentence beginning is defined by the sentenceStartToken
    newSentence = [word2index[sentenceStartToken]]
    # from this, we predict until we get sentenceEndToken
    while not newSentence[-1] == word2index[sentenceEndToken]:
        # that's the main part - here we use the model to predict
        # the next words, given the current state
        nextWordProbabilities,_ = model.forwardProp(newSentence)
        # from this, we will sample a random word
        sampledWord = word2index[unknownToken]
        # iterate through list until we hit word that is not unknown
        while sampledWord == word2index[unknownToken]:
            samples = np.random.multinomial(1, nextWordProbabilities[-1])
            sampledWord = np.argmax(samples)
        newSentence.append(sampledWord)
    # now convert the index into strings
    sentence = [index2word[x] for x in newSentence[1:-1]]
    return sentence

In [ ]:
np.random.seed(1)
# Train on a small subset of the data to see what happens
modelBig = RNN(vocabularySize)
losses = train(modelBig, Xtrain[:10000], ytrain[:10000], nEpoch=50, evaluateLossEvery=5)

In [ ]:
numSentences = 10
sentenceMinLength = 3
 
for i in range(numSentences):
    sentence = []
    # We want long sentences, not sentences with one or two words
    while len(sentence) <= sentenceMinLength:
        sentence = generateSentence(modelBig)
    print(" ".join(sentence))

## Interpreting the results

Example sentences do not look too promising, even though training proceeded for quite some time.

In fact, this implementation does not suffer from computational resources (i.e., simply training longer would be better), but from a very fundamental issue.



# Reading in characters instead
Our model is based on word prediction. What about predicting the next character instead?

Let's try:

In [ ]:
# reading Shakespeare as full text
allData = open('data/shakespeare_plays.txt', 'r').read()
# splitting into characters done smartly making the use of sets/lists
characters = list(set(allData.lower()))
# checking the size of the data
dataSize = len(allData)
vocabularySize = len(characters)
print('Shakespeare has {:d} characters in total with {:d} unique characters in the vocabulary.'.format(dataSize, vocabularySize))
# creating dictionaries of indices and converting back
character2Index = { ch:i for i,ch in enumerate(characters) }
index2character = { i:ch for i,ch in enumerate(characters) }

In [ ]:
# make the training data
p=0
# this will hold the sentences for training and testing 
XtrainChar=list()
ytrainChar=list()
# this is the number of characters we want to investigate
scope = 25
# now go through our data
while p+scope+1<=dataSize:
    # grab the current scope
    i = [character2Index[ch] for ch in allData[p:p+scope].lower()]
    XtrainChar.append(i)
    # offset the scope by one
    t = [character2Index[ch] for ch in allData[p+1:p+scope+1].lower()]
    ytrainChar.append(t)
    p=p+scope

In [ ]:
np.random.seed(1)
# Train on a small subset of the data to see what happens
modelChar = RNN(vocabularySize)
losses = train(modelChar, XtrainChar[:1000], ytrainChar[:1000], nEpoch=200, evaluateLossEvery=1,learningRate=0.001)

In [ ]:
pred=modelChar.predict(XtrainChar[10])
for idx,p in enumerate(pred):
    print("p:",index2character[p],"  wanted:",index2character[ytrainChar[10][idx]])

In [ ]:
probs,_=modelChar.forwardProp(x)

In [ ]:
x=[character2Index[ch] for ch in ['']]
string=list()
string.append(index2character[1])
for i in range(100):
    probs,_=modelChar.forwardProp(x)
    idx = np.random.choice(np.arange(84),1,p=probs[0])
    x.append(idx[0])
    string.append(index2character[idx[0]])
print(''.join(string))